# Digital Hologram Processing with OpenUC2 HoloBox

## Educational Notebook for Understanding Holographic Imaging

This notebook provides an interactive educational experience for understanding digital holography, specifically using the OpenUC2 HoloBox system. We'll explore the physics behind hologram formation, the mathematics of Fresnel propagation, and implement real-time hologram processing using the HoloBox camera API.

### Learning Objectives:
1. Understand the physics of holographic imaging
2. Learn about Fresnel propagation and diffraction
3. Implement digital hologram reconstruction algorithms
4. Explore parameter effects on hologram quality
5. Connect theory to practice with real camera data

### Prerequisites:
- Basic understanding of optics and wave physics
- Python programming knowledge
- Familiarity with NumPy and basic signal processing concepts

## 1. Theory: What is Digital Holography?

### Basic Principles

Digital holography is a technique that captures and reconstructs the complex wavefront (both amplitude and phase) of light waves. Unlike conventional photography which only records intensity, holography preserves the complete optical information.

### Key Concepts:

1. **Wavefront**: A surface of constant phase in a propagating wave
2. **Interference**: When two waves overlap, they create an interference pattern
3. **Diffraction**: The bending of waves around obstacles or through apertures
4. **Fresnel Propagation**: Mathematical description of how light waves propagate through free space

### The Holography Process:

1. **Recording**: Object wave interferes with reference wave, creating interference pattern (hologram)
2. **Reconstruction**: Illuminate hologram to reconstruct original wavefront
3. **Digital Reconstruction**: Use computers to numerically simulate wave propagation

### Mathematical Foundation: Fresnel Diffraction

The complex amplitude U(x,y,z) at distance z from a source plane is given by the Fresnel diffraction integral:

$U(x,y,z) = \frac{e^{ikz}}{i\lambda z} \iint U_0(\xi,\eta) e^{\frac{ik}{2z}[(x-\xi)^2+(y-\eta)^2]} d\xi d\eta$

Where:
- $k = 2\pi/\lambda$ (wave number)
- $\lambda$ is the wavelength
- $U_0(\xi,\eta)$ is the initial complex amplitude
- $(x,y,z)$ are the observation coordinates
- $(\xi,\eta)$ are the source coordinates

## 2. Setup: Import Libraries and Dependencies

First, let's import all necessary libraries for our hologram processing pipeline.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time
import warnings
warnings.filterwarnings('ignore')

# Set up matplotlib for interactive plotting
plt.style.use('default')
%matplotlib widget

print("📚 Libraries imported successfully!")
print("🎯 Ready to explore digital holography!")

## 3. Configuration: Camera API Connection

Configure the connection to the HoloBox camera API. Make sure your HoloBox server is running on the specified address.

In [ ]:
# Camera API Configuration
# Default to localhost - modify this if your HoloBox is on a different IP
API_BASE_URL = "http://localhost:8000"

# Test camera connection
def test_camera_connection(base_url):
    """Test connection to the camera API"""
    try:
        response = requests.get(f"{base_url}/", timeout=5)
        if response.status_code == 200:
            data = response.json()
            print(f"✅ Connected to {base_url}")
            print(f"📷 Camera available: {data.get('camera_available', 'Unknown')}")
            return True
    except requests.exceptions.RequestException as e:
        print(f"❌ Connection failed: {e}")
        print(f"💡 Make sure the camera server is running on {base_url}")
        return False

# Create URL input widget
url_input = widgets.Text(
    value=API_BASE_URL,
    description='API URL:',
    style={'description_width': 'initial'}
)

test_button = widgets.Button(
    description="Test Connection",
    button_style='info'
)

connection_output = widgets.Output()

def on_test_click(b):
    global API_BASE_URL
    with connection_output:
        clear_output()
        API_BASE_URL = url_input.value.rstrip('/')
        test_camera_connection(API_BASE_URL)

test_button.on_click(on_test_click)

display(widgets.VBox([
    widgets.HTML("<h4>🔗 Camera API Connection</h4>"),
    url_input,
    test_button,
    connection_output
]))

# Test initial connection
test_camera_connection(API_BASE_URL)

## 4. Theory Deep Dive: Fresnel Propagation Implementation

Now let's implement the core mathematics of Fresnel propagation. This is the heart of digital hologram reconstruction.

### The Algorithm:

1. **Input**: Complex amplitude field E₀(x,y) at z=0
2. **Fourier Transform**: Convert to frequency domain
3. **Apply Fresnel Kernel**: Multiply by propagation phase factor
4. **Inverse Transform**: Convert back to spatial domain
5. **Output**: Propagated field E(x,y,z)

### Physical Parameters:

- **Wavelength (λ)**: Determines diffraction characteristics
- **Pixel Size**: Spatial sampling resolution
- **Distance (z)**: Propagation distance from hologram to reconstruction plane


In [ ]:
class HologramProcessor:
    """Digital Hologram Processing Class
    
    This class implements the mathematical algorithms for digital hologram reconstruction
    using Fresnel propagation theory.
    """
    
    def __init__(self, wavelength=440e-9, pixel_size=1.4e-6):
        """
        Initialize the hologram processor
        
        Parameters:
        wavelength (float): Light wavelength in meters (default: 440nm blue light)
        pixel_size (float): Camera pixel size in meters (default: 1.4μm)
        """
        self.wavelength = wavelength
        self.pixel_size = pixel_size
        self.wavenumber = 2 * np.pi / wavelength
        
    def abssqr(self, x):
        """Calculate intensity (what a detector sees)
        
        Intensity = |E|² = E × E*
        This is what cameras actually measure - the square of the amplitude.
        """
        return np.real(x * np.conj(x))
    
    def forward_fft(self, x):
        """Forward Fourier transform with proper frequency shift
        
        The fftshift operations ensure that zero frequency is at the center,
        which is the standard convention in optics.
        """
        return np.fft.fftshift(np.fft.fft2(x))
    
    def inverse_fft(self, x):
        """Inverse Fourier transform with proper frequency shift"""
        return np.fft.ifft2(np.fft.ifftshift(x))
    
    def fresnel_propagator(self, E0, distance):
        """
        Fresnel free-space propagation
        
        This is the core algorithm that simulates how light waves propagate
        through free space from the hologram plane to the reconstruction plane.
        
        Parameters:
        E0 (complex array): Initial complex field at z=0
        distance (float): Propagation distance in meters
        
        Returns:
        complex array: Propagated field at distance z
        """
        # Get image dimensions
        height, width = E0.shape
        
        # Physical size of the image
        grid_size_x = self.pixel_size * width
        grid_size_y = self.pixel_size * height
        
        # Create frequency coordinate arrays
        # These represent spatial frequencies in the Fourier domain
        fx = np.linspace(-(width-1)/2 * (1/grid_size_x), 
                        (width-1)/2 * (1/grid_size_x), width)
        fy = np.linspace(-(height-1)/2 * (1/grid_size_y), 
                        (height-1)/2 * (1/grid_size_y), height)
        
        # Create 2D frequency grid
        Fx, Fy = np.meshgrid(fx, fy)
        
        # Fresnel kernel (transfer function)
        # This represents the phase changes that occur during propagation
        H = np.exp(1j * self.wavenumber * distance) * \
            np.exp(1j * np.pi * self.wavelength * distance * (Fx**2 + Fy**2))
        
        # Apply the propagation:
        # 1. Transform to frequency domain
        E0_fft = self.forward_fft(E0)
        
        # 2. Multiply by Fresnel kernel
        G = H * E0_fft
        
        # 3. Transform back to spatial domain
        Ef = self.inverse_fft(G)
        
        return Ef
    
    def process_hologram(self, image_array, distance, crop_size=256):
        """
        Complete hologram processing pipeline
        
        Parameters:
        image_array (numpy array): Input hologram image
        distance (float): Reconstruction distance in meters
        crop_size (int): Size to crop image for faster processing
        
        Returns:
        tuple: (original_intensity, reconstructed_intensity, complex_field)
        """
        # Convert to grayscale if needed
        if len(image_array.shape) == 3:
            gray = np.mean(image_array, axis=2)
        else:
            gray = image_array.copy()
        
        # Normalize to [0,1]
        gray = gray / np.max(gray)
        
        # Crop to square for processing (FFT works best with square, power-of-2 sizes)
        height, width = gray.shape
        crop_size = min(crop_size, min(height, width))
        
        start_y = (height - crop_size) // 2
        start_x = (width - crop_size) // 2
        cropped = gray[start_y:start_y + crop_size, start_x:start_x + crop_size]
        
        # Estimate complex amplitude from intensity
        # In real holography, both amplitude and phase are encoded in the interference pattern
        # Here we approximate by taking the square root of intensity
        amplitude = np.sqrt(cropped)
        
        # Propagate using Fresnel diffraction
        propagated_field = self.fresnel_propagator(amplitude, distance)
        
        # Calculate reconstructed intensity
        reconstructed_intensity = self.abssqr(propagated_field)
        
        return cropped, reconstructed_intensity, propagated_field

# Create the processor instance
processor = HologramProcessor()

print("🔬 Hologram processor initialized!")
print(f"λ = {processor.wavelength*1e9:.0f} nm")
print(f"Pixel size = {processor.pixel_size*1e6:.1f} μm")
print(f"Wave number k = {processor.wavenumber:.0f} rad/m")

## 5. Demonstration: Synthetic Hologram

Before working with real camera data, let's create a synthetic hologram to understand how the parameters affect reconstruction.

In [ ]:
def create_synthetic_hologram(size=256, wavelength=440e-9, pixel_size=1.4e-6, distance=0.005):
    """
    Create a synthetic hologram for demonstration
    
    This simulates what a hologram might look like by creating some objects
    at different distances and computing their interference pattern.
    """
    # Create coordinate grids
    x = np.linspace(-size//2, size//2-1, size) * pixel_size
    y = np.linspace(-size//2, size//2-1, size) * pixel_size
    X, Y = np.meshgrid(x, y)
    
    # Create some simple objects (circular apertures)
    object1 = ((X - 20e-6)**2 + (Y - 10e-6)**2) < (15e-6)**2  # Circle 1
    object2 = ((X + 15e-6)**2 + (Y + 20e-6)**2) < (10e-6)**2  # Circle 2
    object3 = ((X)**2 + (Y - 25e-6)**2) < (8e-6)**2           # Circle 3
    
    # Combine objects
    objects = object1.astype(float) + 0.7*object2.astype(float) + 0.5*object3.astype(float)
    
    # Add some random noise to simulate realistic conditions
    noise = 0.1 * np.random.random((size, size))
    
    # Create the synthetic hologram
    synthetic_hologram = objects + noise
    
    return synthetic_hologram

# Create and display synthetic hologram
synthetic = create_synthetic_hologram()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(synthetic, cmap='gray')
axes[0].set_title('Synthetic Hologram (Input)')
axes[0].axis('off')

# Process the synthetic hologram
original, reconstructed, complex_field = processor.process_hologram(synthetic, distance=0.005)

axes[1].imshow(reconstructed, cmap='hot')
axes[1].set_title('Reconstructed Image')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("🎨 Synthetic hologram demonstration complete!")
print("Notice how the reconstruction reveals the original objects with enhanced contrast.")

## 6. Interactive Parameter Exploration

Now let's create interactive controls to see how different parameters affect hologram reconstruction. This helps build intuition about the physics involved.

In [ ]:
# Interactive parameter exploration
class InteractiveHologramExplorer:
    def __init__(self):
        self.processor = HologramProcessor()
        self.current_image = None
        
        # Create parameter sliders
        self.wavelength_slider = widgets.FloatSlider(
            value=440,
            min=380,
            max=700,
            step=10,
            description='Wavelength (nm):',
            style={'description_width': 'initial'}
        )
        
        self.pixel_size_slider = widgets.FloatSlider(
            value=1.4,
            min=0.5,
            max=5.0,
            step=0.1,
            description='Pixel Size (μm):',
            style={'description_width': 'initial'}
        )
        
        self.distance_slider = widgets.FloatSlider(
            value=5.0,
            min=0.1,
            max=20.0,
            step=0.1,
            description='Distance (mm):',
            style={'description_width': 'initial'}
        )
        
        # Output widget for plots
        self.output = widgets.Output()
        
        # Set up callbacks
        self.wavelength_slider.observe(self.update_plot, names='value')
        self.pixel_size_slider.observe(self.update_plot, names='value')
        self.distance_slider.observe(self.update_plot, names='value')
        
        # Initialize with synthetic data
        self.current_image = create_synthetic_hologram()
        
    def update_processor_parameters(self):
        """Update processor with current slider values"""
        wavelength = self.wavelength_slider.value * 1e-9  # Convert nm to m
        pixel_size = self.pixel_size_slider.value * 1e-6  # Convert μm to m
        
        self.processor = HologramProcessor(wavelength=wavelength, pixel_size=pixel_size)
        
    def update_plot(self, change=None):
        """Update the reconstruction plot with new parameters"""
        if self.current_image is None:
            return
            
        self.update_processor_parameters()
        distance = self.distance_slider.value * 1e-3  # Convert mm to m
        
        with self.output:
            clear_output(wait=True)
            
            # Process hologram
            original, reconstructed, _ = self.processor.process_hologram(
                self.current_image, distance
            )
            
            # Create plot
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            # Original hologram
            axes[0].imshow(original, cmap='gray')
            axes[0].set_title('Input Hologram')
            axes[0].axis('off')
            
            # Reconstructed intensity
            im1 = axes[1].imshow(reconstructed, cmap='hot')
            axes[1].set_title('Reconstructed Intensity')
            axes[1].axis('off')
            plt.colorbar(im1, ax=axes[1], fraction=0.046)
            
            # Phase information (if complex field available)
            phase = np.angle(self.processor.fresnel_propagator(
                np.sqrt(original), distance
            ))
            im2 = axes[2].imshow(phase, cmap='hsv')
            axes[2].set_title('Phase Information')
            axes[2].axis('off')
            plt.colorbar(im2, ax=axes[2], fraction=0.046)
            
            plt.tight_layout()
            plt.show()
            
            # Show current parameters
            print(f"📊 Current Parameters:")
            print(f"λ = {self.wavelength_slider.value:.0f} nm")
            print(f"Pixel size = {self.pixel_size_slider.value:.1f} μm")
            print(f"Distance = {self.distance_slider.value:.1f} mm")
            print(f"Fresnel number = {(self.pixel_size_slider.value*1e-6 * 256)**2 / (4 * self.wavelength_slider.value*1e-9 * self.distance_slider.value*1e-3):.2f}")
    
    def display(self):
        """Display the interactive widget"""
        controls = widgets.VBox([
            widgets.HTML("<h4>🎛️ Interactive Parameter Controls</h4>"),
            widgets.HTML("<p>Adjust the parameters below to see how they affect hologram reconstruction:</p>"),
            self.wavelength_slider,
            self.pixel_size_slider,
            self.distance_slider,
            widgets.HTML("<p><em>The Fresnel number determines the regime of diffraction. Values around 1 give optimal reconstruction.</em></p>")
        ])
        
        display(widgets.HBox([controls, self.output]))
        
        # Initial plot
        self.update_plot()
    
    def set_image(self, image):
        """Set a new image for processing"""
        self.current_image = image
        self.update_plot()

# Create and display the interactive explorer
explorer = InteractiveHologramExplorer()
explorer.display()

## 7. Live Camera Integration

Now let's integrate with the real HoloBox camera to process live holograms! This section connects theory to practice.

In [ ]:
class LiveCameraProcessor:
    def __init__(self, api_base_url):
        self.api_base_url = api_base_url
        self.processor = HologramProcessor()
        self.explorer = explorer  # Use the existing explorer
        
    def capture_snapshot(self):
        """Capture a single frame from the camera"""
        try:
            response = requests.get(f"{self.api_base_url}/snapshot", timeout=10)
            if response.status_code == 200:
                # Convert JPEG bytes to numpy array
                image = Image.open(io.BytesIO(response.content))
                image_array = np.array(image)
                return image_array
            else:
                print(f"❌ Failed to capture: HTTP {response.status_code}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"❌ Network error: {e}")
            return None
    
    def get_camera_stats(self):
        """Get current camera statistics"""
        try:
            response = requests.get(f"{self.api_base_url}/stats", timeout=5)
            if response.status_code == 200:
                return response.json()
        except:
            pass
        return None
    
    def set_camera_settings(self, exposure_us=None, gain=None):
        """Set camera exposure and gain"""
        settings = {}
        if exposure_us is not None:
            settings['exposure_us'] = int(exposure_us)
        if gain is not None:
            settings['gain'] = float(gain)
        
        if settings:
            try:
                response = requests.post(
                    f"{self.api_base_url}/settings",
                    json=settings,
                    timeout=5
                )
                return response.status_code == 200
            except:
                return False
        return True

# Create live processor instance
live_processor = LiveCameraProcessor(API_BASE_URL)

# Camera control widgets
capture_button = widgets.Button(
    description="📸 Capture & Process",
    button_style='success',
    layout=widgets.Layout(width='200px')
)

continuous_button = widgets.Button(
    description="🔄 Start Continuous",
    button_style='info',
    layout=widgets.Layout(width='200px')
)

exposure_slider = widgets.IntSlider(
    value=10000,
    min=100,
    max=100000,
    step=1000,
    description='Exposure (μs):',
    style={'description_width': 'initial'}
)

gain_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Gain:',
    style={'description_width': 'initial'}
)

camera_output = widgets.Output()

# Global variable for continuous processing
continuous_processing = False

def on_capture_click(b):
    """Handle single capture"""
    with camera_output:
        clear_output()
        print("📸 Capturing image...")
        
        # Set camera settings
        live_processor.set_camera_settings(
            exposure_us=exposure_slider.value,
            gain=gain_slider.value
        )
        
        # Capture image
        image = live_processor.capture_snapshot()
        
        if image is not None:
            print(f"✅ Captured {image.shape} image")
            
            # Get camera stats
            stats = live_processor.get_camera_stats()
            if stats:
                print(f"📊 Image stats - Min: {stats['min']}, Max: {stats['max']}, Mean: {stats['mean']:.1f}")
            
            # Process with current explorer parameters
            explorer.set_image(image)
        else:
            print("❌ Failed to capture image")

def on_continuous_click(b):
    """Handle continuous processing toggle"""
    global continuous_processing
    continuous_processing = not continuous_processing
    
    if continuous_processing:
        b.description = "⏹️ Stop Continuous"
        b.button_style = 'warning'
        
        # Start continuous processing
        import threading
        
        def continuous_loop():
            while continuous_processing:
                with camera_output:
                    clear_output(wait=True)
                    print(f"🔄 Continuous processing... (frame at {time.strftime('%H:%M:%S')})")
                    
                    image = live_processor.capture_snapshot()
                    if image is not None:
                        explorer.set_image(image)
                    
                    time.sleep(1)  # Wait 1 second between captures
        
        thread = threading.Thread(target=continuous_loop, daemon=True)
        thread.start()
        
    else:
        b.description = "🔄 Start Continuous"
        b.button_style = 'info'
        with camera_output:
            clear_output()
            print("⏹️ Continuous processing stopped")

# Set up button callbacks
capture_button.on_click(on_capture_click)
continuous_button.on_click(on_continuous_click)

# Display camera controls
camera_controls = widgets.VBox([
    widgets.HTML("<h4>📷 Live Camera Controls</h4>"),
    widgets.HTML("<p>Capture images from the HoloBox camera and process them in real-time:</p>"),
    widgets.HBox([capture_button, continuous_button]),
    widgets.HTML("<h5>Camera Settings:</h5>"),
    exposure_slider,
    gain_slider,
    camera_output
])

display(camera_controls)

print("📷 Live camera integration ready!")
print("💡 Use the controls above to capture and process real holograms")

## 8. Advanced Analysis: Understanding Diffraction Regimes

Let's explore different diffraction regimes and their characteristics. This helps understand when holographic reconstruction works best.

In [ ]:
def analyze_diffraction_regimes():
    """Analyze different diffraction regimes and their characteristics"""
    
    # Create test parameters
    wavelengths = [400e-9, 500e-9, 600e-9, 700e-9]  # Different colors
    distances = [0.001, 0.005, 0.01, 0.02]  # Different distances
    aperture_size = 50e-6  # 50 μm aperture
    pixel_size = 1.4e-6
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    for i, (wavelength, distance) in enumerate(zip(wavelengths, distances)):
        # Calculate Fresnel number
        fresnel_number = aperture_size**2 / (4 * wavelength * distance)
        
        # Create processor for this wavelength
        proc = HologramProcessor(wavelength=wavelength, pixel_size=pixel_size)
        
        # Create a simple aperture
        size = 256
        x = np.linspace(-size//2, size//2-1, size) * pixel_size
        y = np.linspace(-size//2, size//2-1, size) * pixel_size
        X, Y = np.meshgrid(x, y)
        
        aperture = (X**2 + Y**2) < (aperture_size/2)**2
        
        # Process
        original, reconstructed, _ = proc.process_hologram(aperture.astype(float), distance)
        
        # Plot original
        axes[0, i].imshow(original, cmap='gray')
        axes[0, i].set_title(f'Input\nλ={wavelength*1e9:.0f}nm')
        axes[0, i].axis('off')
        
        # Plot reconstruction
        axes[1, i].imshow(reconstructed, cmap='hot')
        axes[1, i].set_title(f'F₁={fresnel_number:.2f}\nz={distance*1000:.1f}mm')
        axes[1, i].axis('off')
    
    plt.suptitle('Diffraction Analysis: Effect of Wavelength and Distance', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Explanation
    print("📊 Diffraction Regime Analysis")
    print("="*50)
    print("Fresnel Number (F₁) = a²/(4λz)")
    print("where a = aperture size, λ = wavelength, z = distance")
    print("")
    print("F₁ >> 1: Geometric optics (sharp shadows)")
    print("F₁ ≈ 1:  Fresnel diffraction (optimal for holography)")
    print("F₁ << 1: Fraunhofer diffraction (far-field)")
    print("")
    print("💡 For best holographic reconstruction, aim for F₁ ≈ 1")

# Run the analysis
analyze_diffraction_regimes()

## 9. Optimization Tips and Best Practices

Here are some practical tips for getting the best results from your holographic imaging system.

In [ ]:
def create_optimization_guide():
    """Create an interactive guide for optimization"""
    
    tips_html = """
    <div style="background: #f0f8ff; padding: 20px; border-radius: 10px; border: 2px solid #4169e1;">
    <h3>🎯 Hologram Optimization Guide</h3>
    
    <h4>📐 Parameter Selection:</h4>
    <ul>
        <li><strong>Wavelength:</strong> Shorter wavelengths (blue) give better resolution but require more precise optics</li>
        <li><strong>Distance:</strong> Adjust to get Fresnel number ≈ 1 for optimal reconstruction</li>
        <li><strong>Pixel Size:</strong> Smaller pixels allow larger reconstruction distances</li>
    </ul>
    
    <h4>📷 Camera Settings:</h4>
    <ul>
        <li><strong>Exposure:</strong> Avoid saturation but get good signal-to-noise ratio</li>
        <li><strong>Gain:</strong> Keep as low as possible to minimize noise</li>
        <li><strong>Focus:</strong> The camera should be slightly out of focus for hologram recording</li>
    </ul>
    
    <h4>🔬 Sample Preparation:</h4>
    <ul>
        <li><strong>Illumination:</strong> Use coherent, collimated light</li>
        <li><strong>Sample:</strong> Semi-transparent objects work best</li>
        <li><strong>Stability:</strong> Minimize vibrations during recording</li>
    </ul>
    
    <h4>💻 Processing Tips:</h4>
    <ul>
        <li><strong>Crop Size:</strong> Use power-of-2 sizes (256, 512) for faster FFT</li>
        <li><strong>Pre-processing:</strong> Apply background subtraction if needed</li>
        <li><strong>Multiple Distances:</strong> Try different reconstruction distances</li>
    </ul>
    
    <h4>🚨 Common Issues:</h4>
    <ul>
        <li><strong>Blurry reconstruction:</strong> Check Fresnel number, adjust distance</li>
        <li><strong>Noise artifacts:</strong> Reduce camera gain, improve illumination</li>
        <li><strong>Low contrast:</strong> Optimize exposure, check sample properties</li>
    </ul>
    </div>
    """
    
    display(HTML(tips_html))
    
    # Parameter calculator
    print("\n🧮 Parameter Calculator")
    print("="*30)
    
    def calculate_optimal_parameters(aperture_size_um, wavelength_nm, target_fresnel=1.0):
        """Calculate optimal distance for given parameters"""
        aperture_size = aperture_size_um * 1e-6
        wavelength = wavelength_nm * 1e-9
        
        optimal_distance = aperture_size**2 / (4 * wavelength * target_fresnel)
        
        print(f"For aperture size: {aperture_size_um} μm")
        print(f"Wavelength: {wavelength_nm} nm")
        print(f"Optimal distance: {optimal_distance*1000:.2f} mm")
        print(f"Fresnel number: {target_fresnel}")
        print("-" * 30)
    
    # Calculate for common scenarios
    calculate_optimal_parameters(50, 440)  # 50μm object, blue light
    calculate_optimal_parameters(100, 532)  # 100μm object, green light
    calculate_optimal_parameters(20, 633)   # 20μm object, red light

create_optimization_guide()

## 10. Exercises and Experiments

Try these exercises to deepen your understanding of digital holography:

In [ ]:
def create_exercises():
    """Create interactive exercises"""
    
    exercises_html = """
    <div style="background: #f5f5dc; padding: 20px; border-radius: 10px; border: 2px solid #daa520;">
    <h3>🧪 Hands-on Exercises</h3>
    
    <h4>Exercise 1: Parameter Sweep</h4>
    <p>Use the interactive controls above to:</p>
    <ol>
        <li>Fix wavelength at 500nm and vary distance from 1-10mm</li>
        <li>Observe how image sharpness changes</li>
        <li>Find the distance that gives the sharpest reconstruction</li>
        <li>Calculate the Fresnel number for optimal distance</li>
    </ol>
    
    <h4>Exercise 2: Wavelength Effects</h4>
    <p>Explore chromatic effects:</p>
    <ol>
        <li>Fix distance at 5mm and pixel size at 1.4μm</li>
        <li>Vary wavelength from 400-700nm</li>
        <li>Notice how reconstruction changes with color</li>
        <li>Explain why blue light gives different results than red</li>
    </ol>
    
    <h4>Exercise 3: Real Sample Analysis</h4>
    <p>If you have access to the camera:</p>
    <ol>
        <li>Place a small transparent object (hair, fiber, etc.) in the beam</li>
        <li>Capture a hologram using the camera controls</li>
        <li>Optimize parameters for best reconstruction</li>
        <li>Try different reconstruction distances</li>
        <li>Compare with direct microscopy if available</li>
    </ol>
    
    <h4>Exercise 4: Phase vs. Amplitude</h4>
    <p>Understanding complex field reconstruction:</p>
    <ol>
        <li>Look at both intensity and phase plots</li>
        <li>Notice how phase reveals additional information</li>
        <li>Try to correlate phase patterns with object features</li>
        <li>Consider what phase tells us that intensity doesn't</li>
    </ol>
    </div>
    """
    
    display(HTML(exercises_html))
    
    # Exercise tracker
    exercise_tracker = widgets.VBox([
        widgets.HTML("<h4>📝 Exercise Progress Tracker</h4>"),
        widgets.Checkbox(description="Exercise 1: Parameter Sweep", value=False),
        widgets.Checkbox(description="Exercise 2: Wavelength Effects", value=False),
        widgets.Checkbox(description="Exercise 3: Real Sample Analysis", value=False),
        widgets.Checkbox(description="Exercise 4: Phase vs. Amplitude", value=False),
    ])
    
    display(exercise_tracker)

create_exercises()

## 11. Summary and Further Reading

### What We've Learned

In this notebook, we've explored:

1. **Theoretical Foundation**: Understanding wave propagation and Fresnel diffraction
2. **Mathematical Implementation**: FFT-based hologram reconstruction algorithms
3. **Parameter Effects**: How wavelength, distance, and pixel size affect reconstruction
4. **Practical Application**: Real-time processing with the HoloBox camera system
5. **Optimization Strategies**: Best practices for high-quality results

### Key Takeaways

- **Holography preserves both amplitude and phase information**
- **Fresnel number F₁ ≈ 1 gives optimal reconstruction quality**
- **Shorter wavelengths provide better resolution but require more precision**
- **Digital methods allow flexible post-processing and analysis**

### Applications

Digital holography is used in:
- Microscopy and cell biology
- Industrial inspection
- Particle tracking and sizing
- Biomedical imaging
- Art conservation

### Further Reading

1. **Books**:
   - "Digital Holography and Digital Image Processing" by Ulf Schnars
   - "Handbook of Digital Holography" by Pascal Picart

2. **Research Papers**:
   - "Digital holographic microscopy: a noninvasive contrast imaging technique"
   - "Fresnel approximation in digital holography"

3. **Online Resources**:
   - OpenUC2 Project: https://github.com/openUC2
   - Digital Holography Community
   - Optics textbooks and courses

### Next Steps

- Experiment with different samples and lighting conditions
- Try advanced reconstruction algorithms (angular spectrum, etc.)
- Explore 3D holographic imaging
- Build your own holographic setup with OpenUC2 components

---

**Thank you for exploring digital holography with the OpenUC2 HoloBox!** 🔬✨

*This notebook demonstrates the power of combining theoretical understanding with practical implementation. Continue experimenting and learning!*